In [1]:
# Importing the Keras libraries and packages
from keras.utils import np_utils
from keras.models import Sequential, load_model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# Importing other necessary libraries
from sklearn.metrics import classification_report,confusion_matrix
from PIL import Image
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import h5py, os, itertools, heapq

%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import keras

c:\users\default.laptop-2ci68m4p\anaconda3\envs\tensorflow1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Architecture

In [2]:
vgg16_model = keras.applications.vgg16.VGG16()
vgg16_model.summary()  #Original VGG16 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [3]:
# Now freeze all top layers, delete last, add own last layer
model = Sequential()
for layer in vgg16_model.layers:
    model.add(layer)

# pop removed last layer (predictions (dense) 1000)
model.layers.pop()  

#freeze the top layers
for layer in model.layers:
    layer.trainable = False

# Add my own 6 class layer
model.add(Dense(units = 6, activation = 'softmax'))  
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

### Image preprocessing

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   rotation_range = 40,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('./dataset/training_set',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('./dataset/validation_set',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 5831 images belonging to 6 classes.
Found 255 images belonging to 6 classes.


### Callbacks (save weights while training)

In [5]:
# Setting callbacks parameters
checkpointer = ModelCheckpoint(filepath='vgg16.{epoch:02d}-{val_loss:.2f}.hdf5', verbose=1, save_best_only=True)
filename='vgg16.csv'
csv_log = CSVLogger(filename, separator=',', append=False)

### Training

In [6]:
# Training the model
hist = model.fit_generator(training_set,
                           steps_per_epoch = (5831//32),
                           epochs = 200,
                           validation_data = test_set,
                           validation_steps = (255//32), 
                           workers = 4, 
                           callbacks = [csv_log, checkpointer])

Epoch 1/200
182/182 [==============================] - 47s 260ms/step - loss: 1.7828 - acc: 0.2117 - val_loss: 1.7821 - val_acc: 0.2143

Epoch 00001: val_loss improved from inf to 1.78211, saving model to vgg16.01-1.78.hdf5
Epoch 2/200
182/182 [==============================] - 42s 232ms/step - loss: 1.7594 - acc: 0.2511 - val_loss: 1.7844 - val_acc: 0.1696

Epoch 00002: val_loss did not improve from 1.78211
Epoch 3/200
182/182 [==============================] - 43s 234ms/step - loss: 1.7213 - acc: 0.2632 - val_loss: 1.7085 - val_acc: 0.2812

Epoch 00003: val_loss improved from 1.78211 to 1.70855, saving model to vgg16.03-1.71.hdf5
Epoch 4/200
182/182 [==============================] - 43s 234ms/step - loss: 1.6891 - acc: 0.2779 - val_loss: 1.7046 - val_acc: 0.2277

Epoch 00004: val_loss improved from 1.70855 to 1.70464, saving model to vgg16.04-1.70.hdf5
Epoch 5/200
182/182 [==============================] - 43s 234ms/step - loss: 1.6519 - acc: 0.2979 - val_loss: 1.6313 - val_acc: 0.3


Epoch 00039: val_loss did not improve from 1.28164
Epoch 40/200
182/182 [==============================] - 43s 235ms/step - loss: 1.3836 - acc: 0.2970 - val_loss: 1.2794 - val_acc: 0.2768

Epoch 00040: val_loss improved from 1.28164 to 1.27937, saving model to vgg16.40-1.28.hdf5
Epoch 41/200
182/182 [==============================] - 43s 236ms/step - loss: 1.3789 - acc: 0.3084 - val_loss: 1.3896 - val_acc: 0.2812

Epoch 00041: val_loss did not improve from 1.27937
Epoch 42/200
182/182 [==============================] - 43s 235ms/step - loss: 1.3745 - acc: 0.3059 - val_loss: 1.4397 - val_acc: 0.2589

Epoch 00042: val_loss did not improve from 1.27937
Epoch 43/200
182/182 [==============================] - 43s 236ms/step - loss: 1.3867 - acc: 0.3046 - val_loss: 1.2807 - val_acc: 0.2902

Epoch 00043: val_loss did not improve from 1.27937
Epoch 44/200
182/182 [==============================] - 43s 237ms/step - loss: 1.3607 - acc: 0.3117 - val_loss: 1.3516 - val_acc: 0.2768

Epoch 00044: v

182/182 [==============================] - 43s 236ms/step - loss: 1.3331 - acc: 0.3049 - val_loss: 1.2560 - val_acc: 0.2991

Epoch 00082: val_loss did not improve from 1.25312
Epoch 83/200
182/182 [==============================] - 43s 234ms/step - loss: 1.3395 - acc: 0.3096 - val_loss: 1.3035 - val_acc: 0.2946

Epoch 00083: val_loss did not improve from 1.25312
Epoch 84/200
182/182 [==============================] - 43s 235ms/step - loss: 1.3327 - acc: 0.3111 - val_loss: 1.3040 - val_acc: 0.2946

Epoch 00084: val_loss did not improve from 1.25312
Epoch 85/200
182/182 [==============================] - 43s 236ms/step - loss: 1.3234 - acc: 0.3061 - val_loss: 1.2507 - val_acc: 0.3125

Epoch 00085: val_loss improved from 1.25312 to 1.25070, saving model to vgg16.85-1.25.hdf5
Epoch 86/200
182/182 [==============================] - 43s 235ms/step - loss: 1.3428 - acc: 0.3108 - val_loss: 1.2643 - val_acc: 0.3571

Epoch 00086: val_loss did not improve from 1.25070
Epoch 87/200
182/182 [======

KeyboardInterrupt: 

# Conclusion

### I had to terminate this training half way, due to extremely low accuracy. No point continuing. Had to look for other solution.
